<a href="https://colab.research.google.com/github/Raunaq14/Data_Sci_Research/blob/main/Combine_FOItext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import pandas as pd
import os
from glob import glob

def process_foitext_year(year):
    print(f"🔄 Processing FOI Text for {year}...")

    zip_path = f"{zip_dir}/foitext{year}.zip"
    output_dir = f"{extract_dir}/foitext{year}"
    os.makedirs(output_dir, exist_ok=True)

    # Unzip files
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
    print(f"📂 Extracted to {output_dir}")

    all_files = glob(f"{output_dir}/*.txt") + glob(f"{output_dir}/*.csv")
    dfs = []

    for file in all_files:
        try:
            # Try default comma-separated
            df = pd.read_csv(file, encoding='latin1', low_memory=False)
            print(f"✅ Loaded with comma: {os.path.basename(file)}")
        except pd.errors.ParserError:
            try:
                # Try pipe-separated
                df = pd.read_csv(file, sep='|', encoding='ISO-8859-1', low_memory=False)
                print(f"✅ Loaded with pipe: {os.path.basename(file)}")
            except Exception as e:
                print(f"❌ Skipped {os.path.basename(file)}: {e}")
                continue
        dfs.append(df)

    if dfs:
        combined_df = pd.concat(dfs, ignore_index=True)
        save_path = f"{final_save_dir}/foitext_{year}.csv"
        combined_df.to_csv(save_path, index=False)
        print(f"✅ Saved combined FOI Text for {year}: {save_path}")
    else:
        print(f"⚠️ No valid files were found for year {year}.")

# Base paths (reused)
zip_dir = "/content/drive/MyDrive/Extracted_FDA_Data/zip"
extract_dir = "/content/drive/MyDrive/Extracted_FDA_Data/extracted_foitext"
final_save_dir = "/content/drive/MyDrive/Extracted_FDA_Data/final_merged_data"
os.makedirs(extract_dir, exist_ok=True)
os.makedirs(final_save_dir, exist_ok=True)

# Run for both years
process_foitext_year(2023)
process_foitext_year(2024)

🔄 Processing FOI Text for 2023...
📂 Extracted to /content/drive/MyDrive/Extracted_FDA_Data/extracted_foitext/foitext2023
⚠️ No valid files were found for year 2023.
🔄 Processing FOI Text for 2024...
📂 Extracted to /content/drive/MyDrive/Extracted_FDA_Data/extracted_foitext/foitext2024
✅ Loaded with pipe: foitext2024.txt
✅ Saved combined FOI Text for 2024: /content/drive/MyDrive/Extracted_FDA_Data/final_merged_data/foitext_2024.csv


In [ ]:
import pandas as pd

# Manually extracted FOI file path
foi_2023_path = "/content/drive/MyDrive/Extracted_FDA_Data/extracted_foitext/foitext2023/foitext2023.txt"
save_path = "/content/drive/MyDrive/Extracted_FDA_Data/final_merged_data/foitext_2023.csv"

# Load using pipe separator
foi_2023_df = pd.read_csv(foi_2023_path, sep='|', encoding='ISO-8859-1', on_bad_lines='skip', low_memory=False)

# Save to CSV
foi_2023_df.to_csv(save_path, index=False)

print("✅ FOI Text 2023 saved to:", save_path)

✅ FOI Text 2023 saved to: /content/drive/MyDrive/Extracted_FDA_Data/final_merged_data/foitext_2023.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

def merge_device_mdr_foi(year):
    print(f"🔄 Merging data for {year}...")

    device_mdr_path = f"/content/drive/MyDrive/Extracted_FDA_Data/final_merged_data/final_data_{year}.csv"
    foi_path = f"/content/drive/MyDrive/Extracted_FDA_Data/final_merged_data/foitext_{year}.csv"
    output_path = f"/content/drive/MyDrive/Extracted_FDA_Data/final_merged_data/final_data_with_foi_{year}.csv"

    # Load datasets
    device_df = pd.read_csv(device_mdr_path, dtype=str, low_memory=False)
    foi_df = pd.read_csv(foi_path, dtype=str, low_memory=False)

    # Check for key column
    if 'MDR_REPORT_KEY' not in device_df.columns or 'MDR_REPORT_KEY' not in foi_df.columns:
        raise KeyError("MDR_REPORT_KEY missing in one of the datasets")

    # Merge on MDR_REPORT_KEY (left join to preserve device+mdr structure)
    merged_df = pd.merge(device_df, foi_df, how='left', on='MDR_REPORT_KEY')

    # Save final merged file
    merged_df.to_csv(output_path, index=False)
    print(f"✅ Merged and saved: {output_path}")

# Run for both years
#merge_device_mdr_foi(2023)
merge_device_mdr_foi(2024)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔄 Merging data for 2024...
✅ Merged and saved: /content/drive/MyDrive/Extracted_FDA_Data/final_merged_data/final_data_with_foi_2024.csv
